In [ ]:

# Install necessary libraries
!pip install -q torch torchvision torchaudio
!pip install -q transformers
!pip install -q torch-geometric
!pip install -q networkx matplotlib
!pip install -q git+https://github.com/salesforce/BLIP.git


In [ ]:

import os
import json
import torch
import torch.nn.functional as F
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration


In [ ]:

# BLIP for VLM image understanding
device = 'cuda' if torch.cuda.is_available() else 'cpu'

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

def describe_board_image(image_path):
    image = Image.open(image_path).convert('RGB')
    inputs = processor(image, return_tensors="pt").to(device)
    caption_ids = blip_model.generate(**inputs)
    return processor.decode(caption_ids[0], skip_special_tokens=True)


In [ ]:

# Load puzzle graph from JSON
def load_graph_from_json(json_path):
    with open(json_path, "r") as f:
        graph_json = json.load(f)

    G = nx.DiGraph()
    for node in graph_json["nodes"]:
        G.add_node(node["id"], label=node.get("label", ""))
    for edge in graph_json["edges"]:
        G.add_edge(edge["source"], edge["target"])
    return G


In [ ]:

def graph_to_data(graph):
    node_mapping = {node: i for i, node in enumerate(graph.nodes)}
    features = []

    for node in graph.nodes:
        label = graph.nodes[node].get('label', '').lower()
        feat = [
            'ball' in label,
            'bit' in label,
            'gear' in label,
            'interceptor' in label,
            'crossover' in label,
            'ramp' in label,
            'toggle' in label,
            'output' in label
        ]
        features.append(torch.tensor(feat, dtype=torch.float))

    edge_index = []
    for src, dst in graph.edges:
        edge_index.append([node_mapping[src], node_mapping[dst]])

    x = torch.stack(features)
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    return Data(x=x, edge_index=edge_index)


In [ ]:

class SymbolicGNN(torch.nn.Module):
    def __init__(self, in_features=8, hidden=32, out_features=8):
        super().__init__()
        self.conv1 = GCNConv(in_features, hidden)
        self.conv2 = GCNConv(hidden, out_features)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x


In [ ]:

class TransformerPlanner:
    def __init__(self):
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.model = GPT2LMHeadModel.from_pretrained("gpt2")

    def generate_plan(self, symbolic_output):
        input_tokens = " ".join([f"{i}:{','.join(map(str, f.tolist()))}" for i, f in enumerate(symbolic_output)])
        inputs = self.tokenizer.encode(input_tokens, return_tensors="pt")
        outputs = self.model.generate(inputs, max_length=100, num_return_sequences=1)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:

class HybridPuzzleAgent:
    def __init__(self):
        self.gnn = SymbolicGNN()
        self.planner = TransformerPlanner()

    def solve(self, graph):
        data = graph_to_data(graph)
        symbolic_output = self.gnn(data.x, data.edge_index)
        return self.planner.generate_plan(symbolic_output)


In [ ]:

# Example run (replace paths with your own uploaded files)
# from google.colab import files
# uploaded = files.upload()

# Load graph
# G = load_graph_from_json("puzzle08.json")
# agent = HybridPuzzleAgent()
# plan = agent.solve(G)
# print("Generated Plan:", plan)
# nx.draw(G, with_labels=True, node_color='lightgreen', edge_color='gray')
# plt.show()
